Integrantes:
- Felipe Galdames.
- Felipe Duran.

# Tarea.
- Usando las funciones anteriores, o desarrollando las propias, escriba una función que pueda valorizar, por simulación de Montecarlo, calls y puts sobre bonos cero cupón con el modelo de HW. Considere puts y calls de plazo máximo igual a 1Y.
- Su función debe poder recibir la semilla.
- Compare sus resultados con las fórmulas del modelo y muestre que diferencias obtiene usando 1000 simulaciones y una semilla predeterminada.
- Valorice también por simulación de Montecarlo el bono cupón cero a 1Y. Muestre que diferencias obtiene respecto al valor de mercado del bono usando 1000 simulaciones y una semilla predeterminada.

## Importamos Las Librerias A Utilizar.

In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
from typing import List, Tuple, Callable
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

#from modules
import auxiliary as aux

from numpy import random as rnd
from scipy.stats import norm
import pandas as pd
import numpy as np
import inspect
import os
import decimal
import plotly.express as px
import math

## Definimos todas las funciones que se utilizarán a lo largo de la tarea, en orden según son llamadas originalmente.

In [2]:
# Se definen las funciones que dan la primera y segunda derivadas del valor de las tasas
def zrate(t: float) -> float:
    return zcurva(t)


def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

In [3]:
# Calcula f(0,t) 
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)
# Calcula f'(0,t)/dt
def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

La función $\theta_{t}$ esta dada por:

$$
\begin{equation}
\theta_{t}=\frac{\partial f(0,t)}{\partial t}+\gamma^*f(0,t)+\frac{\sigma^2}{2\gamma^*}\left[1-\exp(-2\gamma^*t)\right]
\end{equation}
$$

Tal como se muestra en la siguiente linea de codigo:

In [4]:
# Calculos de Theta.
# Funcion
def theta(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

In [5]:
def sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed = None):
    """
    """
    dt = 1 / 264.0
    num_steps += 1
    
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    
    # Se rellenan los instantes de tiempo
    for i in range(0, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta(i * dt)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return tiempo, sim

In [6]:
# Codigo B
def b_hw(gamma: float, t: float, T: float) -> float:
    """
    Calcula el valor de la función B(t,T) 

    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

A, se calcula de la forma: 
$$
\begin{equation}
A\left(t,T\right)=\log\frac{Z\left(r_0,0,T\right)}{Z\left(r_0,0,t\right)}+B\left(t,T\right)f\left(0,t\right)-\frac{\sigma^2}{4\gamma^*}B\left(t,T\right)^2\left[1-\exp\left(-2\gamma^* T\right)\right)]
\end{equation}
$$

In [7]:
# Codigo de A

def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3


El valor de un Bono Cero Cupon esta dado por:

$$
\begin{equation}
Z\left(r_t,t,T\right)=\exp\left[A\left(t,T\right)-B\left(t,T\right)r_t\right]
\end{equation}
$$


In [8]:
# Valorizacion cupon
def zero_hw(r: float, gamma: float, sigma: float,
            zrate: float, fwd: float, t: float, T: float) -> float:
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)


Hemos definido todas las funciones, por lo que procederemos a indicar las fórmulas asociadas al modelo.

El modelo sigue un proceso estocastico dado de la siguiente forma:
    
$$
\begin{equation}
dr_t=\left(\theta_t-\gamma^* r_t\right)dt+\sigma dX_t
\end{equation}
$$

donde la función $\theta_{t}$ esta dada por:

$$
\begin{equation}
\theta_{t}=\frac{\partial f(0,t)}{\partial t}+\gamma^*f(0,t)+\frac{\sigma^2}{2\gamma^*}\left[1-\exp(-2\gamma^*t)\right]
\end{equation}
$$

$$
\frac{\partial f\left(0,t\right)}{\partial t}=\frac{\partial^2 r\left(0,t\right) }{\partial t^2}t+2\frac{\partial r\left(0,t\right) }{\partial t}
$$

$$f(0,t)=r(0,t)+t\frac{\partial r(0,t)}{\partial t}$$

$$\frac{\partial f(0,t)}{\partial t}=2\frac{\partial r(0,t)}{\partial t}+t\frac{\partial^2r(0,t)}{\partial t^2}$$

## Procedemos a cargar la data y a procesarla.

In [9]:
# Data con la que se trabaja. 
curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
frmt = {'tasa': '{:.4%}', 'df': '{:.6%}'}

# Se calcula el tiempo y tasa de forma continua
curva['t'] = curva['plazo'] / 365.0
curva['rate'] = np.log(1 / curva['df'])/( curva['plazo'] / 365.0)

zcurva = interp1d(curva['t'],
                  curva['rate'],
                  kind='cubic',
                  fill_value="extrapolate")

curva.head(100).style.format(frmt)

,plazo,tasa,df,t,rate
0,1,0.0811%,99.999778%,0.002740,0.000811
1,7,0.0841%,99.998388%,0.019178,0.000841
2,14,0.0780%,99.997010%,0.038356,0.000780
3,21,0.0774%,99.995549%,0.057534,0.000774
4,33,0.0781%,99.992942%,0.090411,0.000781
5,61,0.0781%,99.986954%,0.167123,0.000781
6,92,0.0811%,99.979560%,0.252055,0.000811
7,125,0.0781%,99.973271%,0.342466,0.000781
8,152,0.0760%,99.968343%,0.416438,0.000760
9,182,0.0750%,99.962603%,0.498630,0.000750


A continuacion se muestra la extrapolacion de todos los datos:

In [10]:
steps_per_year = 365
years = 1
dt = 1 / steps_per_year
result = []
for i in range(0, steps_per_year * years):
    result.append((i * dt, zcurva(i * dt)))
df_result = pd.DataFrame(result, columns=['plazo', 'tasa'])
df_result

,plazo,tasa
0,0.000000,0.0007881405316711811
1,0.002740,0.0008111102098831921
2,0.005479,0.0008278198168317605
3,0.008219,0.0008389276473914464
4,0.010959,0.000845091996436812
...,...,...
360,0.986301,0.0007034755095980428
361,0.989041,0.0007032583236912854
362,0.991781,0.0007030399443477396
363,0.994521,0.00070282038501222


Recordar que la función $\theta_{t}$ esta dada por:

$$
\begin{equation}
\theta_{t}=\frac{\partial f(0,t)}{\partial t}+\gamma^*f(0,t)+\frac{\sigma^2}{2\gamma^*}\left[1-\exp(-2\gamma^*t)\right]
\end{equation}
$$

lo cual se muestra en la siguiente linea de codigo:

In [11]:
# Parametros iniciales
sigma = .015
gamma = .5
r0 = zrate(0)

# Probamos que la funcion sirva
t=1
print(f"thetha({t:.3f})={theta(t):.5%}")

thetha(1.000)=0.01386%


# Codigo Simulaciones de Montecarlo.

Nuevamente se muestra el codigo para realizar simulaciones de montecarlo.

In [12]:
def sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed = None):
    """
    """
    dt = 1 / 264.0
    num_steps += 1
    
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    
    # Se rellenan los instantes de tiempo
    for i in range(0, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta(i * dt)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return tiempo, sim

# Bono Cupon Zero.

A continuacion se valorizara un bono con las formulas de HW. Se calcula B(t,T) de la forma:

$$
\begin{equation}
B\left(t,T\right)=\frac{1}{\gamma^*}\left[1-\exp\left(-\gamma^* T\right)\right)]
\end{equation}
$$


In [13]:
print(f"r0:        {r0:.8%}")
t = 0 # tiempo 0
T = 1 # 1 año
z = zero_hw(r0, gamma, sigma, zrate, fwd, t, T)
print(f"Valorizacion formulas HW:    {z:.4%}")
print(f"Valorizacion curva: {math.exp(-zrate(T) * T):.4%}")

r0:        0.07881405%
Valorizacion formulas HW:    99.9298%
Valorizacion curva: 99.9298%


## Bono Cupon Zero por MonteCarlo.

A continuacion se valoriza el mismo Bono Cero Cupon por Monte-Carlo utilizando 1000 simulaciones. Al final se comparan los resultados obtenidos con Monte-Carlo, la formula de HW y la interpolacion.

In [14]:
# Numero de simulaciones y pasos
num_sim = 1000
num_steps = 263
print(f"num_steps: {num_steps}")
seed = 7
df = 0

# Obtenemos los tiempos y trayectorias de las num_sim simulaciones.
tiempo, s = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)

# Obtenemos los factores de descuento. 
for sim in s:
    df += math.exp(np.sum(-dt * np.sum(sim)))

ez = df / num_sim
z_curva = math.exp(-zrate(T) * T)
print(f"El valor con MonteCarlo es: {ez: .5}")
print(f"Valorizacion formula HW: {z: .5}")
print(f"Valorizacion curva: {z_curva: .5}")


num_steps: 263
El valor con MonteCarlo es:  0.99947
Valorizacion formula HW:  0.9993
Valorizacion curva:  0.9993


# Valorizacion de Put's y Call's.

Utilizaremos los siguientes parametros iniciales.

In [15]:
# Parametros iniciales
sigma = .015
gamma = .5
r0 = zrate(0)
To=1
Tb=2
t=0
#T=1
K=1
r0=zrate(0)

print(f"r_0= {r0:.4%}")
print(f"T_0= {To}")
print(f"T_B= {Tb}")

r_0= 0.0788%
T_0= 1
T_B= 2


In [16]:
ZTo=zero_hw(r0, gamma, sigma, zrate, fwd, t, To)
ZTb=zero_hw(r0, gamma, sigma, zrate, fwd, t, Tb)
print(ZTo)
print(ZTb)

0.9992978688293874
0.9988654872527635


Calculamos:
$$
\begin{equation}
B\left(t,T\right)=\frac{1}{\gamma^*}\left[1-\exp\left(-\gamma^* T\right)\right)]
\end{equation}
$$
Luego se calcula:
$$
\begin{equation}
S_Z\left(T_O\right)=B\left(T_O,T_B\right)\sqrt{\frac{\sigma^2}{2\gamma^*}\left(1-\exp\left(-2\gamma^* T_O\right)\right)}
\end{equation} 
$$

In [17]:
b=b_hw(gamma, To, Tb)
Sz=b*math.sqrt( (sigma**2)*(1-math.exp(-2*gamma*To))/(2*gamma))
print(f"B(T_0,T_B)= {b}")
print(f"S_z(T_0)= {Sz}")

B(T_0,T_B)= 0.7869386805747332
S_z(T_0)= 0.009384953162988194


Luego se calcula $d_1$ y $d_2$ de la forma:

$$
\begin{equation}
d_1=\frac{\log\left(\frac{Z\left(r_0,0,T_B\right)}{KZ\left(r_0,0,T_O\right)}\right)+\frac{1}{2}S_Z\left(T_O\right)^2}{\sqrt{S_Z\left(T_O\right)}}
\end{equation}
$$


$$
\begin{equation}
d_2=d_1-S_Z\left(T_O\right)
\end{equation}
$$

In [18]:

d1=(math.log(ZTb/(ZTo*K))+0.5*Sz**2)/Sz
d2=d1-Sz
print(d1)
print(d2)

-0.04142166019983301
-0.050806613362821204


Finalmente se calcula el precio de la opcion Call en este caso de la forma:

$$
\begin{equation}
Call\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(d_1\right)-KN\left(d_2\right)\right]
\end{equation}
$$

In [19]:

Call=ZTo*(ZTb/ZTo*norm.cdf(d1)-K*norm.cdf(d2))
print(f'El precio de la opcion Call es: {Call}')

El precio de la opcion Call es: 0.0035283883401179164


$$
\begin{equation}
Put\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[KN\left(-d_2\right)-\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(-d_1\right)\right]
\end{equation}
$$

In [20]:
Put=ZTo*(-ZTb/ZTo*norm.cdf(-d1)+K*norm.cdf(-d2))
print(f'El precio de la opcion Put es: {Put}')

El precio de la opcion Put es: 0.003960769916741911


### Test

In [21]:
import hull_white as hw

In [22]:
hw.zcb_call_put(
    hw.CallPut.CALL,
    strike=K,
    to=To,
    tb=Tb,
    r0=r0,
    zo=math.exp(-zrate(To) * To),
    zb=math.exp(-zrate(Tb) * Tb),
    gamma=gamma,
    sigma=sigma)

0.0035283883401179272

In [23]:
hw.zcb_call_put(
    hw.CallPut.PUT,
    strike=K,
    to=To,
    tb=Tb,
    r0=r0,
    zo=math.exp(-zrate(To) * To),
    zb=math.exp(-zrate(Tb) * Tb),
    gamma=gamma,
    sigma=sigma)

0.00396076991674188

## Simulaciones con Monte-Carlo de opciones Call y Put.

In [24]:
# Numero de simulaciones y pasos
num_sim = 1000
num_steps = 263
print(f"num_steps: {num_steps}")
seed = 7
df = 0

# Obtenemos los tiempos y trayectorias de las num_sim simulaciones.
tiempo, s = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)
fDescuentos=[]


# Obtenemos los factores de descuento. 
for sim in s:
    df += math.exp(np.sum(-dt * np.sum(sim)))
    fDescuentos.append(math.exp(-dt*np.sum(sim)))

    
last_rates = [sim[-1] for sim in s]    
precios = [zero_hw(r, gamma, sigma, zrate, fwd, t=1, T=2) * 1 for r in last_rates]

ez = df / num_sim
z_curva = math.exp(-zrate(T) * T)
print(f"El valor con MonteCarlo es: {ez: .5}")
print(f"Valorizacion formula HW: {z: .5}")
print(f"Valorizacion curva: {z_curva: .5}")

num_steps: 263
El valor con MonteCarlo es:  0.99947
Valorizacion formula HW:  0.9993
Valorizacion curva:  0.9993


In [25]:
callMC=[]
putMC=[]
for i in range(0,1000):
    callMC.append(max(precios[i]-1,0)*fDescuentos[i])
    putMC.append(max(1-precios[i],0)*fDescuentos[i])


print(f'El valor de la opcion Call con Monte-Carlo es: {np.mean(callMC)}')
print(f'El valor de la opcion Put con Monte-Carlo es: {np.mean(putMC)}')

El valor de la opcion Call con Monte-Carlo es: 0.003499826538082184
El valor de la opcion Put con Monte-Carlo es: 0.003998163380897726


**AD:** falta conexión y explicación entre las distintas partes del notebook. Se repite código.